# 📋 1. 환경 설정 

## 1.1. 필요한 라이브러리 설치 

In [ ]:
# pip 업그레이드
%pip install --quiet --upgrade pip

# Agent Framework 설치 (약 3-4분 소요)
%pip install --quiet agent-framework==1.0.0b260130 --pre

# 설치된 agent-framework 버전 확인
%pip show agent-framework

> 최초 설치시 상단에 있는 Kernel 메뉴에서 "Restart Kernel" 버튼을 눌러서 다시 시작해줍니다.

## 1.2: Azure 인증 설정

**인증 방식 선택 (둘 중 하나):**
- `USE_API_KEY = True`: API Key 기반 인증 (엔드포인트 + API Key)
- `USE_CLI_CREDENTIAL = True`: `az login` 후 CLI 토큰 사용

아래 셀에서 원하는 방식을 `True`로 설정하세요.

In [ ]:
import os
from azure.identity import AzureCliCredential

# ============================================
# 🔐 인증 방식 선택 (둘 중 하나만 True로 설정!)
# ============================================
USE_API_KEY = False          # API Key 기반 인증
USE_CLI_CREDENTIAL = True    # AzureCliCredential 사용 (az login 필요)

# ============================================

if USE_API_KEY and USE_CLI_CREDENTIAL:
    raise ValueError("❌ 둘 중 하나만 True로 설정하세요!")

if not USE_API_KEY and not USE_CLI_CREDENTIAL:
    raise ValueError("❌ 최소 하나는 True로 설정하세요!")

if USE_API_KEY:
    # API Key 인증 - 아래 값을 본인 정보로 수정!
    AZURE_OPENAI_API_KEY = "your-api-key-here"  # 👈 API Key 입력
    
    credential = None  # API Key 사용 시 credential 불필요
    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    
    print("🔐 API Key 인증 사용")
    print("   (API Key로 Azure OpenAI에 직접 연결)")
    
    if AZURE_OPENAI_API_KEY == "your-api-key-here":
        print("\n⚠️ API Key를 실제 값으로 수정하세요!")
    else:
        print("\n✅ API Key 설정 완료!")

elif USE_CLI_CREDENTIAL:
    credential = AzureCliCredential()
    print("🔐 AzureCliCredential 사용")
    print("   (터미널에서 'az login' 필요)")
    
    # 인증 테스트
    try:
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print("\n✅ Azure CLI 인증 성공!")
    except Exception as e:
        print(f"\n❌ 인증 실패: {e}")
        print("   터미널에서 'az login'을 실행하세요.")

## 1.2. 환경변수 설정

In [ ]:
import os

#  아래 값을 본인의 Azure OpenAI 정보로 수정하세요!
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ms-foundry-1023.openai.azure.com/"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4.1"  # 또는 gpt-4.1

print("✅ Azure OpenAI 설정 완료!")
print(f"   Endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}")
print(f"   Deployment: {os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME']}")

# 2. Concurrent Orchestration 구현

이 노트북은 Microsoft Agent Framework의 공식 문서를 기반으로 작성되었습니다.

📖 **참고 문서**: [Concurrent Orchestration - Microsoft Learn](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/concurrent?pivots=programming-language-python)

Concurrent orchestration 은 여러 에이전트가 동일한 작업을 병렬로 수행할 수 있도록 합니다. 각 에이전트는 입력을 독립적으로 처리하며, 그 결과는 이후 수집되고 종합됩니다. 이 방식은 브레인스토밍, 앙상블 추론, 투표 시스템처럼 다양한 관점이나 여러 해법이 중요한 시나리오에 특히 적합합니다.

<img src="https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/resources/images/orchestration-concurrent.png" width="80%" height="80%">

## 2.1. 개별 도메인에 특화된 에이전트 생성

에이전트는 특정 작업을 처리하도록 설계된 전문화된 개체입니다. 아래 코드는 연구 전문가, 마케팅 전문가, 그리고 법률 전문가의 세 가지 에이전트를 정의합니다.

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 1) AzureChatClient를 사용해 3개의 도메인 에이전트 생성
chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

researcher = chat_client.as_agent(
    instructions=(
        "당신은 시장 및 제품 조사 전문가입니다. "
        "프롬프트가 주어지면 간결하고 사실 기반의 인사이트, "
        "기회 요인, 그리고 리스크를 제공합니다."
    ),
    name="researcher",
)

marketer = chat_client.as_agent(
    instructions=(
        "당신은 창의적인 마케팅 전략가입니다. "
        "프롬프트에 맞춰 설득력 있는 가치 제안과 "
        "타깃 메시지를 설계합니다."
    ),
    name="marketer",
)

legal = chat_client.as_agent(
    instructions=(
        "당신은 신중한 법무·컴플라이언스 검토자입니다. "
        "프롬프트를 기반으로 제약 사항, 면책 조항, "
        "정책 및 규제 관련 이슈를 식별합니다."
    ),
    name="legal",
)


## 2.2. Concurrent Orchestration 생성

ConcurrentBuilder 클래스는 여러 에이전트를 병렬로 실행하는 워크플로우를 구성할 수 있도록 해줍니다. 이때 에이전트 목록을 참가자(participants)로 전달합니다.

In [ ]:
from agent_framework import ConcurrentBuilder

# 2) 동시 실행(Concurrent) 워크플로우 구성
# participants에는 Agent(AgentProtocol 타입) 또는 Executor를 전달할 수 있음
workflow = (
    ConcurrentBuilder()
    .participants([researcher, marketer, legal])
    .build()
)

## 2.3. Concurrent Workflow 실행

In [ ]:
from typing import Any
from agent_framework import ChatMessage, WorkflowOutputEvent

# 3) 단일 프롬프트로 워크플로우 실행
#    실행 중 진행 상황을 스트리밍하고, 최종으로 합쳐진 메시지를 보기 좋게 출력
output_evt: WorkflowOutputEvent | None = None

async for event in workflow.run_stream(
    "도심 통근자를 위한 새로운 가성비 전기 자전거를 출시할 예정입니다."
):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

if output_evt:
    print("===== 최종 통합 대화 결과 (메시지) =====")
    messages: list[ChatMessage] | Any = output_evt.data

    for i, msg in enumerate(messages, start=1):
        name = msg.author_name if msg.author_name else "user"
        print(
            f"{'-' * 60}\n\n"
            f"{i:02d} [{name}]:\n"
            f"{msg.text}"
        )


# 3. Group Chat Orchestration

이 노트북은 Microsoft Agent Framework의 공식 문서를 기반으로 작성되었습니다.

📖 **참고 문서**: [Group Chat Orchestration - Microsoft Learn](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/group-chat?pivots=programming-language-python)

Group chat orchestration은 여러 에이전트가 함께 협업하는 대화를 모델링하며, 대화의 흐름과 다음에 발언할 에이전트를 결정하는 오케스트레이터가 이를 조정합니다. 이 패턴은 반복적인 개선이 필요하거나, 협업 기반 문제 해결, 또는 다양한 관점을 통한 분석이 필요한 시나리오에 특히 적합합니다.

내부적으로 group chat orchestration은 중앙에 오케스트레이터가 위치한 별(star) 형태의 토폴로지로 에이전트들을 구성합니다. 오케스트레이터는 라운드 로빈 방식, 프롬프트 기반 선택, 또는 대화 맥락에 따른 사용자 정의 로직 등 다양한 전략을 통해 다음에 발언할 에이전트를 선택할 수 있어, 멀티 에이전트 협업을 위한 유연하면서도 강력한 패턴을 제공합니다.

<img src="https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/resources/images/orchestration-groupchat.png" width="80%" height="80%">

### Group Chat과 다른 패턴 간의 차이점

Group chat orchestration은 다른 멀티 에이전트 패턴과 비교했을 때 다음과 같은 뚜렷한 특징을 가집니다.

- 중앙 집중식 조율(Centralized Coordination): 에이전트 간에 직접 제어를 넘기는 handoff 패턴과 달리, group chat에서는 오케스트레이터가 다음에 발언할 에이전트를 결정합니다.

- 반복적 개선(Iterative Refinement): 에이전트들이 여러 라운드에 걸쳐 서로의 응답을 검토하고, 보완하며, 발전시킬 수 있습니다.

- 유연한 발언자 선택(Flexible Speaker Selection): 오케스트레이터는 라운드 로빈, 프롬프트 기반 선택, 또는 사용자 정의 로직 등 다양한 전략을 활용해 발언자를 선택할 수 있습니다.

- 공유 컨텍스트(Shared Context): 모든 에이전트가 전체 대화 이력을 공유하므로, 협업 기반의 정교한 응답 개선이 가능합니다.

## 3.1. 개별 도메인에 특화된 에이전트 생성

에이전트는 특정 작업을 처리하도록 설계된 전문화된 개체입니다. 아래 코드는 연구가(Researcher), 작가(Writer) 전문가의 두가지 에이전트를 정의합니다.

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework import ChatAgent

# Initialize the Azure OpenAI chat client
chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

# 리서처 에이전트 생성
researcher = ChatAgent(
    name="Researcher",
    description="질문에 답하는 데 필요한 배경 정보를 수집합니다.",
    instructions="질문에 도움이 되는 핵심 사실을 간결하게 수집하세요. 짧고 사실 위주로 작성하세요.",
    chat_client=chat_client,
)

# 작성자 에이전트 생성
writer = ChatAgent(
    name="Writer",
    description="수집된 정보를 바탕으로 정제된 답변을 작성합니다.",
    instructions="제공된 메모나 정보를 활용해 명확하고 구조적인 답변을 작성하세요. 충분히 상세하게 설명하세요.",
    chat_client=chat_client,
)


## 3.2. Group Chat Orchestration 생성

지능적인 발언자 선택을 위해 에이전트 기반 오케스트레이터를 사용합니다. 이 오케스트레이터는 도구, 공유된 컨텍스트, 그리고 관측 가능성(Observability)에 접근할 수 있는 ChatAgent입니다.

In [ ]:
from agent_framework import GroupChatBuilder, GroupChatState, Role

# 발언자 선택을 담당하는 오케스트레이터 에이전트 생성
orchestrator_agent = ChatAgent(
    name="Orchestrator",
    description="발언자를 선택하여 멀티 에이전트 협업을 조율합니다.",
    instructions="""
당신은 사용자 요청을 해결하기 위해
에이전트 팀의 대화를 조율하는 역할을 합니다.

가이드라인:
- 먼저 리서처가 정보를 수집하도록 합니다.
- 그 다음 작성자가 최종 답변을 종합하도록 합니다.
- 최종 답변이 충분하면 오케스트레이터가 작성자에게 받은 내용을 그대로 전달합니다.
""",
    chat_client=chat_client,
)

# 에이전트 기반 오케스트레이터를 사용하는 Group Chat 구성
workflow = (
    GroupChatBuilder()
    .with_orchestrator(agent=orchestrator_agent)
    # 하드 종료 조건 설정: Assistant 메시지가 4개 이상이면 종료
    # 일반적으로는 오케스트레이터가 그 이전에 지능적으로 종료를 결정합니다.
    .with_termination_condition(
        lambda messages: sum(
            1 for msg in messages if msg.role == Role.ASSISTANT
        ) >= 4
    )
    .participants([researcher, writer])
    .build()
)


## 3.3. Group Chat Workflow 실행

In [ ]:
from typing import cast
from agent_framework import WorkflowOutputEvent, ChatMessage
from agent_framework import AgentRunUpdateEvent

task = "Python에서 async/await를 사용하는 핵심적인 장점은 무엇인가요?"

print(f"Task: {task}\n")
print("=" * 80)

final_conversation: list[ChatMessage] = []
last_executor_id: str | None = None

# Run the workflow
async for event in workflow.run_stream(task):
    # print(type(event))
    if isinstance(event, AgentRunUpdateEvent):
        # Print streaming agent updates
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            print(f"[{eid}]:", end=" ", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        # Workflow completed - data is a list of ChatMessage
        final_conversation = cast(list[ChatMessage], event.data)

if final_conversation:
    print("\n\n" + "=" * 80)
    print("Final Conversation:")
    for msg in final_conversation:
        author = getattr(msg, "author_name", "Unknown")
        text = getattr(msg, "text", str(msg))
        print(f"\n[{author}]\n{text}")
        print("-" * 80)

print("\nWorkflow completed.")